<a href="https://colab.research.google.com/github/KotaroT1/KotaroT1/blob/main/Diabetes_Prediction_Challenge_Kaggle_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Diabetes Prediction Challenge

In [1]:
# Google Driveへのマウント
# 実行するとGoogleログイン画面が出るため、コンペデータを格納しているアカウントでログインしてください

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 必要モジュールでColab環境にないものはinstall
!pip install japanize_matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 22.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for japanize_matplotlib: filename=japanize_matplotlib-1.1.3-py3-none-any.whl size=4120257 sha256=44fd5cc7e8ceed418ccdd362317291b44d3c0813cd161beafb476c6b208498e8
  Stored in directory: /root/.cache/pip/wheels/c1/f7/9b/418f19a7b9340fc16e071e89efc379aca68d40238b258df53d
Successfully built japanize_matplotlib


In [3]:
# 必要モジュールをimport
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib_venn import venn2
import seaborn as sns
import japanize_matplotlib
import lightgbm as lgb
from sklearn.model_selection import train_test_split, KFold, GroupKFold
from sklearn.metrics import make_scorer
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore')

# ランダム性を伴う処理を行うため、結果の再現性を保つにはシード値を固定しておく必要があります
SEED = 42
np.random.seed(SEED)

In [4]:
# 表示できるdfの行、列数を増やす
pd.set_option("display.max_rows", 100)    # 最大100行まで表示
pd.set_option("display.max_columns", 100) # 最大100列まで表示

In [5]:
# Google Driveがマウントされているか確認し、base_pathを設定
if os.path.exists('/content/drive/My Drive'):
    # Google Driveがマウントされている場合
    # ご自身の環境に合わせて下記を適切なパスに変更してください
    base_path = '/content/drive/MyDrive/Colab Notebooks/Kaggle_DiabetesPredictionChallenge/data'
else:
    # Google Driveがマウントされていない場合
    base_path = './'

print(f"データ読み込み元パス: {base_path}")


# データ読み込み
# player_idやmatch_idの数値的大小に意味はないのでstring形式で読み込み
train_df = pd.read_csv(f"{base_path}/train.csv")
test_df = pd.read_csv(f"{base_path}/test.csv")
submission_df = pd.read_csv(f"{base_path}/sample_submission.csv")

print(f"trainデータ形状: {train_df.shape}")
display(train_df.head(3))
print(f"\ntestデータ形状: {test_df.shape}")
display(test_df.head(3))

データ読み込み元パス: /content/drive/MyDrive/Colab Notebooks/Kaggle_DiabetesPredictionChallenge/data
trainデータ形状: (700000, 26)


,id,age,alcohol_consumption_per_week,physical_activity_minutes_per_week,diet_score,sleep_hours_per_day,screen_time_hours_per_day,bmi,waist_to_hip_ratio,systolic_bp,diastolic_bp,heart_rate,cholesterol_total,hdl_cholesterol,ldl_cholesterol,triglycerides,gender,ethnicity,education_level,income_level,smoking_status,employment_status,family_history_diabetes,hypertension_history,cardiovascular_history,diagnosed_diabetes
0,0,31,1,45,7.7,6.8,6.1,33.4,0.93,112,70,62,199,58,114,102,Female,Hispanic,Highschool,Lower-Middle,Current,Employed,0,0,0,1.0
1,1,50,2,73,5.7,6.5,5.8,23.8,0.83,120,77,71,199,50,121,124,Female,White,Highschool,Upper-Middle,Never,Employed,0,0,0,1.0
2,2,32,3,158,8.5,7.4,9.1,24.1,0.83,95,89,73,188,59,114,108,Male,Hispanic,Highschool,Lower-Middle,Never,Retired,0,0,0,0.0



testデータ形状: (300000, 25)


,id,age,alcohol_consumption_per_week,physical_activity_minutes_per_week,diet_score,sleep_hours_per_day,screen_time_hours_per_day,bmi,waist_to_hip_ratio,systolic_bp,diastolic_bp,heart_rate,cholesterol_total,hdl_cholesterol,ldl_cholesterol,triglycerides,gender,ethnicity,education_level,income_level,smoking_status,employment_status,family_history_diabetes,hypertension_history,cardiovascular_history
0,700000,45,4,100,4.3,6.8,6.2,25.5,0.84,123,70,64,209,55,135,111,Female,White,Highschool,Middle,Former,Employed,0,0,0
1,700001,35,1,87,3.5,4.6,9.0,28.6,0.88,120,74,59,159,47,83,145,Female,White,Highschool,Middle,Never,Unemployed,0,0,0
2,700002,45,1,61,7.6,6.8,7.0,28.5,0.94,112,71,75,173,43,99,184,Male,White,Highschool,Low,Never,Employed,0,0,0


In [ ]:
y = train_df['diagnosed_diabetes']
X = train_df.drop(['id', 'diagnosed_diabetes'], axis=1)

# KFoldでデータを分割
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=SEED)

print(f"データを{n_splits}分割します。")

In [9]:
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder

# カテゴリカルな特徴量をリストアップ
categorical_features = [
    'gender', 'ethnicity', 'education_level', 'income_level',
    'smoking_status', 'employment_status'
]

# Label Encodingを適用
for col in categorical_features:
    le = LabelEncoder()
    # train_dfとtest_dfの両方に存在するカテゴリを全て学習させるため、結合してからfitする
    # 存在しないカテゴリがある場合は、エラーを避けるためにNaNにする
    combined_data = pd.concat([X[col], test_df[col]], axis=0).astype(str).unique()
    le.fit(combined_data)

    X[col] = le.transform(X[col].astype(str))
    test_df[col] = le.transform(test_df[col].astype(str))

# モデル、OOF予測、テスト予測のリストを初期化
models = []
oof_preds = np.zeros(len(X))
test_preds = np.zeros(len(test_df))

# KFoldでデータをループ処理
for fold, (train_index, val_index) in enumerate(kf.split(X, y)):
    print(f"\nFold {fold+1}/{n_splits} 開始")

    # 訓練データと検証データを分割
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]

    # LightGBMモデルの初期化と学習
    lgb_clf = lgb.LGBMClassifier(
        objective='binary',  # 二値分類
        metric='auc',        # 評価指標としてAUCを使用
        random_state=SEED,
        n_estimators=1000,
        learning_rate=0.05,
        num_leaves=31,
        max_depth=-1,
        n_jobs=-1,
        colsample_bytree=0.8,
        subsample=0.8,
        reg_alpha=0.1,
        reg_lambda=0.1,
    )

    lgb_clf.fit(X_train, y_train,
                eval_set=[(X_val, y_val)],
                eval_metric='auc',
                callbacks=[lgb.early_stopping(stopping_rounds=100, verbose=False)])

    # 検証データに対する予測（確率）
    val_preds = lgb_clf.predict_proba(X_val)[:, 1]
    oof_preds[val_index] = val_preds

    # テストデータに対する予測（確率）
    # 各フォールドの予測を合計し、最終的に平均を取る
    test_preds += lgb_clf.predict_proba(test_df.drop('id', axis=1))[:, 1] / n_splits

    # モデルを保存
    models.append(lgb_clf)

    # このフォールドのAUCを計算
    fold_auc = roc_auc_score(y_val, val_preds)
    print(f"Fold {fold+1} AUC: {fold_auc:.4f}")

# OOF予測全体のAUCを計算
overall_oof_auc = roc_auc_score(y, oof_preds)
print(f"\nOOF全体のAUC: {overall_oof_auc:.4f}")


Fold 1/5 開始
[LightGBM] [Info] Number of positive: 348936, number of negative: 211064
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.160833 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1644
[LightGBM] [Info] Number of data points in the train set: 560000, number of used features: 24
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.623100 -> initscore=0.502727
[LightGBM] [Info] Start training from score 0.502727
Fold 1 AUC: 0.7265

Fold 2/5 開始
[LightGBM] [Info] Number of positive: 348863, number of negative: 211137
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.292876 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1641
[LightGBM] [Info] Number of data points in the train set: 560000, number of used features: 24
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.622970 -> initscore=0.502172
[LightGBM] [Info] 

最終的な予測結果を提出形式に整えます。

In [17]:
submission_df['diagnosed_diabetes'] = test_preds
display(submission_df.head())

# 結果をCSVファイルとして保存
submission_df.to_csv('submission.csv', index=False)
print("提出ファイル 'submission.csv' を作成しました。")

,id,diagnosed_diabetes
0,700000,0.504374
1,700001,0.685584
2,700002,0.768794
3,700003,0.380056
4,700004,0.911489


提出ファイル 'submission.csv' を作成しました。


In [18]:
submission_df.to_csv(f"{base_path}/DPC_submissions_01.csv", index=False)

テスト
